# 50인 데이터 매칭 알고리즘

사용 언어: Python

가치관 질문은 사용하지 않는다고 가정하고 우선 제외한 상태로 작성했습니다.

현재 구현 버전에서는 성별/연령 차이를 구현하지 않았습니다.

(09/09 수정: 매칭 알고리즘 보완)

## 데이터 정제

데이터 분석 및 정리에 필요한 라이브러리(pandas, numpy)를 불러옵니다.

주어진 엑셀 파일에서 미리 사용하지 않을 열(타임스탬프, 가치관 질문, 빈 열 등)을 삭제하고, CSV 파일형식으로 변환하여 불러옵니다.
dataCount 변수에 현재 데이터 개수(응답 인원수)를 저장합니다.

In [1]:
import pandas as pd
import numpy as np

# set of column names
headerNames = ["q1", "q2", "q3", "q4", "q5", "q6", "q7","q8", "q9", "q10", "gender", "age", "polAlign"]

# read csv-converted data from excel file
# current data is cleaned to exclude empty and/or meaningless columns
answerData = pd.read_csv('data/50data_answeronly.csv', header =0, names=headerNames)

# save number of entries
dataCount = len(answerData.index)

answerData

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,gender,age,polAlign
0,③ 중립,① 매우 찬성,② 찬성,④ 반대,② 찬성,① 매우 찬성,③ 중립,② 찬성,③ 중립,② 찬성,남,40,2
1,① 매우 찬성,③ 중립,② 찬성,② 찬성,⑤ 매우 반대,④ 반대,② 찬성,④ 반대,③ 중립,① 매우 찬성,여,25,2
2,② 찬성,⑤ 매우 반대,③ 중립,② 찬성,① 매우 찬성,④ 반대,② 찬성,③ 중립,⑤ 매우 반대,② 찬성,남,31,3
3,⑤ 매우 반대,② 찬성,④ 반대,⑤ 매우 반대,① 매우 찬성,④ 반대,① 매우 찬성,① 매우 찬성,⑤ 매우 반대,③ 중립,남,51,4
4,③ 중립,③ 중립,③ 중립,③ 중립,② 찬성,③ 중립,③ 중립,③ 중립,③ 중립,③ 중립,여,38,3
5,② 찬성,④ 반대,② 찬성,④ 반대,② 찬성,④ 반대,② 찬성,④ 반대,② 찬성,③ 중립,여,47,3
6,① 매우 찬성,④ 반대,② 찬성,③ 중립,⑤ 매우 반대,① 매우 찬성,⑤ 매우 반대,② 찬성,① 매우 찬성,② 찬성,여,21,2
7,② 찬성,① 매우 찬성,② 찬성,② 찬성,④ 반대,③ 중립,③ 중립,② 찬성,③ 중립,① 매우 찬성,남,32,3
8,④ 반대,② 찬성,⑤ 매우 반대,④ 반대,④ 반대,④ 반대,④ 반대,① 매우 찬성,① 매우 찬성,① 매우 찬성,여,52,4
9,⑤ 매우 반대,③ 중립,⑤ 매우 반대,⑤ 매우 반대,① 매우 찬성,② 찬성,② 찬성,② 찬성,④ 반대,① 매우 찬성,남,56,5


비수치화 되어 있는 응답 결과를 숫자 형태의 데이터값으로 변환해 줍니다.

In [2]:
# replace all redundant text to interger values

answerData.replace(["① 매우 찬성","② 찬성", "③ 중립", "④ 반대", "⑤ 매우 반대", "남", "여"], [1, 2, 3, 4, 5, "M", "F"], inplace = True)

answerData

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,gender,age,polAlign
0,3,1,2,4,2,1,3,2,3,2,M,40,2
1,1,3,2,2,5,4,2,4,3,1,F,25,2
2,2,5,3,2,1,4,2,3,5,2,M,31,3
3,5,2,4,5,1,4,1,1,5,3,M,51,4
4,3,3,3,3,2,3,3,3,3,3,F,38,3
5,2,4,2,4,2,4,2,4,2,3,F,47,3
6,1,4,2,3,5,1,5,2,1,2,F,21,2
7,2,1,2,2,4,3,3,2,3,1,M,32,3
8,4,2,5,4,4,4,4,1,1,1,F,52,4
9,5,3,5,5,1,2,2,2,4,1,M,56,5


## 응답 유사도 측정

모든 개인과 개인 간의 응답 유사도를 저장할 (데이터 수)^2 사이즈의 행렬인 diffMatrix를 새로 만듭니다.
기본적으로는 NaN (값이 없는 상태)로 초기화합니다.

In [3]:
# create a new dataframe to save values
# this will be very time and cost extensive, might be viable to change this part
# will be durable for several hundred calculations though...?

diffMatrix = pd.DataFrame(np.nan, index=range(dataCount), columns=range(dataCount), dtype='float')

각 개인과 개인의 응답 결과를 비교하여, 각 질문에 대한 총 응답 차이값 = Sum(|개별 응답 차|) 값을 저장합니다.
i번째 응답자와 j번째 응답자의 응답 차이값은 diffMatrix의 i행 j열에 저장됩니다. (j행 i열의 값과 동일합니다.)
자기 자신과의 응답차는 계산하지 않도록 짜 두어, 대각선으로 NaN 값이 나옵니다.

(50인 데이터에서는 수 초 내에 결과가 나오지만, 실제 데이터에서는 응답차 값을 계산하는 것에 상당한 시간이 소요될 것으로 예상됩니다.
 현재 모델에서는 개인과 개인 간의 총 응답차 값을 모두 계산해야 하기 때문에 이 부분이 필수적이라는 점 또한 유의해 주시기 바랍니다.)

In [4]:
for i in range(dataCount):
    for j in range(i, dataCount):
        tempVal=0;
        if (i==j):
            diffMatrix.iloc[i][j] = np.nan
            continue
        for k in range(10):
            # iterates through 10 for the number of questions in the dataframe
            tempVal += answerData.iloc[i][k]-answerData.iloc[j][k]
        # Assign same value to the flipped index to save computation time
        diffMatrix.iloc[i][j] = abs(tempVal)
        diffMatrix.iloc[j][i] = abs(tempVal)
        
diffMatrix

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,NaN,4.0,6.0,8.0,6.0,6.0,3.0,0.0,7.0,7.0,...,5.0,24.0,19.0,1.0,0.0,4.0,7.0,11.0,7.0,9.0
1,4.0,NaN,2.0,4.0,2.0,2.0,1.0,4.0,3.0,3.0,...,1.0,20.0,15.0,3.0,4.0,0.0,3.0,7.0,11.0,5.0
2,6.0,2.0,NaN,2.0,0.0,0.0,3.0,6.0,1.0,1.0,...,1.0,18.0,13.0,5.0,6.0,2.0,1.0,5.0,13.0,3.0
3,8.0,4.0,2.0,NaN,2.0,2.0,5.0,8.0,1.0,1.0,...,3.0,16.0,11.0,7.0,8.0,4.0,1.0,3.0,15.0,1.0
4,6.0,2.0,0.0,2.0,NaN,0.0,3.0,6.0,1.0,1.0,...,1.0,18.0,13.0,5.0,6.0,2.0,1.0,5.0,13.0,3.0
5,6.0,2.0,0.0,2.0,0.0,NaN,3.0,6.0,1.0,1.0,...,1.0,18.0,13.0,5.0,6.0,2.0,1.0,5.0,13.0,3.0
6,3.0,1.0,3.0,5.0,3.0,3.0,NaN,3.0,4.0,4.0,...,2.0,21.0,16.0,2.0,3.0,1.0,4.0,8.0,10.0,6.0
7,0.0,4.0,6.0,8.0,6.0,6.0,3.0,NaN,7.0,7.0,...,5.0,24.0,19.0,1.0,0.0,4.0,7.0,11.0,7.0,9.0
8,7.0,3.0,1.0,1.0,1.0,1.0,4.0,7.0,NaN,0.0,...,2.0,17.0,12.0,6.0,7.0,3.0,0.0,4.0,14.0,2.0
9,7.0,3.0,1.0,1.0,1.0,1.0,4.0,7.0,0.0,NaN,...,2.0,17.0,12.0,6.0,7.0,3.0,0.0,4.0,14.0,2.0


diffMatrix 내에 저장되어 있는 모든 값들의 중간값을 diffMedian 변수에 저장합니다.
50인 테스트 데이터셋의 경우, 중간값은 5.0에서 형성되었습니다.

In [5]:
# Calculate the median value of all the diff values (currently, it's 5)
diffMedian = diffMatrix.stack().median()

print(diffMedian)

5.0


## 매칭 알고리즘

매칭을 통해 만들어진 짝 정보를 저장할 수 있는 빈 array를 만듭니다.
같은생각 매칭은 samePairs에, 다른생각 매칭은 diffPairs에 저장합니다.

In [6]:
# set of arrays to save the created pairs

samePairs = []
diffPairs = []

매칭된 횟수를 별도로 저장할 array를 만들고(초기값은 0입니다), 주어진 규칙에 따라 0열부터 매칭을 진행합니다.
현재 찾은 응답차 최소인 대상이 이미 3번 이상 매치가 되었을 때, 그 값을 무시하고 3회 매칭에 성공하거나,
응답차가 diffMedian 값을 초과할 경우, 이미 모든 응답차 값이 중앙값 이상이기 때문에 매칭을 취소합니다.

(09/09 수정: 0열부터 매칭을 진행, 매칭이 될 경우 지속적으로 매칭을 시도하지 않습니다.
상호 매칭을 허용하고, 매칭의 유사도 순위를 함께 출력합니다. (매칭 결과 상 먼저 나오는 응답자 기준))

In [7]:
# create separate array to denote how many times it has been assigned to pairs
matchSame = [0]*dataCount

for i in range(dataCount):
    levelCount = 1
    matched = False
    # Shallow copy the current row to mutate
    tempArray = list(diffMatrix[i]) #split array to prevent double matches
    tempArray[i] = 100
    
    while(not matched):
        tempSame = np.argmin(tempArray) # Return mutated index of the current minumum diff individual
        if (tempArray[tempSame] > diffMedian):
            # if the current min value is larger than the median Diff, stop searching  
            break
        elif (matchSame[i] >= 3):
            break
            # if the match count for the current row is already over 3, stop searching
        elif (matchSame[tempSame] < 3):
            # if the current match count for both items are less than 3, provide match
            samePairs.append([i, tempSame, levelCount])

            matchSame[i] += 1
            matchSame[tempSame] +=1

            tempArray[tempSame] = 100
            # replace tempArray value to a large number that will not be min-ed
            
            matched = True;
        else:
            levelCount += 1
            tempArray[tempSame] = 100
            
print(samePairs)
print(matchSame)

[[0, 7, 1], [1, 23, 1], [2, 4, 1], [3, 8, 1], [4, 2, 1], [5, 2, 1], [6, 1, 1], [7, 0, 1], [8, 9, 1], [9, 8, 1], [10, 0, 1], [11, 37, 1], [12, 9, 2], [13, 16, 1], [14, 7, 2], [15, 4, 2], [16, 13, 1], [17, 18, 1], [18, 17, 1], [19, 13, 1], [20, 12, 3], [21, 17, 1], [22, 25, 1], [23, 1, 1], [24, 35, 1], [25, 48, 1], [26, 47, 1], [27, 10, 3], [28, 5, 3], [29, 5, 3], [30, 25, 1], [31, 15, 4], [32, 15, 4], [33, 18, 2], [34, 26, 1], [35, 24, 1], [36, 23, 2], [37, 11, 1], [38, 34, 1], [39, 11, 1], [40, 24, 1], [41, 42, 1], [42, 38, 1], [43, 37, 2], [44, 10, 3], [45, 36, 3], [46, 12, 3], [47, 26, 1], [48, 22, 2], [49, 3, 1]]
[3, 3, 3, 2, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 1, 3, 2, 3, 3, 1, 1, 1, 2, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1]


같은 방식으로 다른생각 매칭도 진행하였습니다.

(09/09 수정: 응답차 순위가 상당히 높게 나오는 )

In [8]:
matchDiff = [0]*dataCount 
   
for i in range(dataCount):
    levelCount = 1
    matched = False
    # Shallow copy the current row to mutate
    tempArray = list(diffMatrix[i]) #split array to prevent double matches
    tempArray[i] = -100 
    
    while (not matched):
        tempDiff = np.argmax(tempArray) # Return mutated index of the current maximum diff individual
        if (tempArray[tempDiff] < diffMedian):
            # if the current max value is smaller than the median Diff, stop searching  
            break
        elif (matchDiff[i] >= 3):
            # if the match count for the current row is already over 3, stop searching
            break
        elif (matchDiff[tempDiff] < 3):
            # if the current match count for both items are less than 3, provide match
            diffPairs.append([i, tempDiff, levelCount])
            
            matchDiff[i] += 1
            matchDiff[tempDiff] +=1
            
            tempArray[tempDiff] = -100
            matched = True
            # replace tempArray value to a small number that will not be max-ed
        else: 
            levelCount += 1
            tempArray[tempDiff] = -100                 
            
print(diffPairs)
print(matchDiff)

[[0, 41, 1], [1, 41, 1], [2, 41, 1], [3, 30, 1], [4, 30, 2], [5, 30, 2], [6, 42, 2], [7, 42, 2], [8, 25, 3], [9, 25, 3], [10, 42, 2], [11, 38, 3], [12, 25, 3], [13, 38, 3], [14, 38, 3], [15, 48, 5], [16, 34, 4], [17, 34, 5], [18, 34, 5], [19, 26, 5], [20, 48, 4], [21, 26, 7], [22, 26, 5], [23, 48, 5], [24, 22, 6], [27, 47, 6], [28, 22, 6], [29, 13, 7], [31, 13, 7], [32, 16, 8], [33, 47, 8], [35, 16, 9], [36, 47, 12], [37, 49, 10], [39, 49, 10], [40, 19, 10], [43, 49, 10], [44, 3, 9], [45, 19, 13], [46, 0, 10]]
[2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 3, 3, 1, 1, 1, 1, 3, 3, 3]
